In [1]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import StratifiedKFold

In [2]:
train=pd.read_csv('../Unbalanced_Dataset.csv',index_col=0)
test_set=pd.read_csv('../testset.csv',index_col=0)

train_copy=train.drop('id',axis=1)
X = train_copy.drop('smoking', axis=1)
y = train_copy['smoking']

test_set=test_set.drop('id',axis=1)
X_test_new= test_set.drop('smoking', axis=1)
y_test_new = test_set['smoking']

In [ ]:
import optuna
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score
import numpy as np

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def objective(trial):
    
    n_d = trial.suggest_int("n_d", 16, 64, step=8)        
    n_a = trial.suggest_int("n_a", 16, 64, step=8)        
    n_steps = trial.suggest_int("n_steps", 4, 10)         
    gamma = trial.suggest_float("gamma", 1.0, 2.0)        
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-4, log=True)  
    lr = trial.suggest_float("lr", 5e-4, 5e-3, log=True)  
    max_epochs = trial.suggest_int("max_epochs", 50, 200) 

    scores = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]   

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_fold)
        X_val_scaled   = scaler.transform(X_val_fold)

        
        X_train_fold = X_train_scaled.astype(np.float32)
        X_val_fold   = X_val_scaled.astype(np.float32)
        y_train_fold = y_train_fold.values.astype(np.int64)
        y_val_fold   = y_val_fold.values.astype(np.int64)

        model = TabNetClassifier(
            n_d=n_d,
            n_a=n_a,
            n_steps=n_steps,
            gamma=gamma,
            lambda_sparse=lambda_sparse,
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=lr),
            verbose=0,
            seed=42
        )

        model.fit(
            X_train_fold, y_train_fold,
            # eval_set=[(X_val_fold, y_val_fold)],
            # eval_metric=['accuracy'],
            max_epochs=max_epochs,
            # patience=15,
            batch_size=1024,
            virtual_batch_size=128
        )

        y_pred_val = model.predict(X_val_fold)
        acc = accuracy_score(y_val_fold, y_pred_val)
        
        scores.append(acc)


    return np.mean(scores)


sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=5)

print("best_params:", study.best_params)
print("best_value:", study.best_value)


[I 2025-08-10 22:29:43,147] A new study created in memory with name: no-name-54e255e4-331d-47bc-bbdc-d7b8ae9d425a



Early stopping occurred at epoch 49 with best_epoch = 34 and best_val_0_accuracy = 0.728


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Stop training because you reached max_epochs = 58 with best_epoch = 48 and best_val_0_accuracy = 0.735


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


...


Early stopping occurred at epoch 69 with best_epoch = 54 and best_val_0_accuracy = 0.76167


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 51 with best_epoch = 36 and best_val_0_accuracy = 0.757


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-08-10 22:45:53,098] Trial 4 finished with value: 0.7548 and parameters: {'n_d': 48, 'n_a': 16, 'n_steps': 8, 'gamma': 1.1705241236872914, 'lambda_sparse': 1.3492834268013243e-06, 'lr': 0.004444833953509465, 'max_epochs': 195}. Best is trial 1 with value: 0.7577999999999999.


best_params: {'n_d': 64, 'n_a': 48, 'n_steps': 8, 'gamma': 1.0205844942958024, 'lambda_sparse': 8.706020878304853e-05, 'lr': 0.003399481210795565, 'max_epochs': 82}
best_value: 0.7577999999999999


In [ ]:

X_train_f, X_val_f, y_train_f, y_val_f = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_f_scaled = scaler.fit_transform(X_train_f)
X_val_f_scaled   = scaler.transform(X_val_f)
X_test_new_scaled  = scaler.transform(X_test_new)


X_train_scaled_np = X_train_f_scaled.astype(np.float32)
y_train_np = y_train_f.values.astype(np.int64)

X_val_scaled_np = X_val_f_scaled.astype(np.float32)
y_val_np = y_val_f.values.astype(np.int64)

X_test_new_scaled_np = X_test_new_scaled.astype(np.float32)
y_test_new_np = y_test_new.values.astype(np.int64)



tabnet_model = TabNetClassifier(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=0.003399481210795565),
    scheduler_params={"step_size":10, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    verbose=10,
    n_d=64, n_a=48, n_steps=8, gamma=1.0205844942958024,
    lambda_sparse=8.706020878304853e-05,
    seed=42
)



tabnet_model.fit(
    X_train=X_train_scaled_np, y_train=y_train_np,
    batch_size=1024,
    virtual_batch_size=128,
    max_epochs=33,
    num_workers=4
)

y_train_pred = tabnet_model.predict(X_train_scaled_np)
train_acc = accuracy_score(y_train_np, y_train_pred)

print(f"TabNet Train Accuracy: {train_acc:.4f}")
# print(classification_report(y_train_np, y_train_pred, digits=4))


y_pred = tabnet_model.predict(X_test_new_scaled_np)
acc = accuracy_score(y_test_new_np, y_pred)

print(f"\n TabNet Test Accuracy: {acc:.4f}")
print(classification_report(y_test_new_np, y_pred, digits=4))

/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 1.02547 |  0:00:25s
epoch 10 | loss: 0.5296  |  0:04:40s
epoch 20 | loss: 0.47922 |  0:08:57s
epoch 30 | loss: 0.47235 |  0:13:13s
TabNet Train Accuracy: 0.7604

 TabNet Test Accuracy: 0.7420
              precision    recall  f1-score   support

           0     0.7093    0.8200    0.7607      1000
           1     0.7867    0.6640    0.7202      1000

    accuracy                         0.7420      2000
   macro avg     0.7480    0.7420    0.7404      2000
weighted avg     0.7480    0.7420    0.7404      2000



In [6]:
tabnet_model.save_model("tabnet_model(1.5w2)")


Successfully saved model at tabnet_model(1.5w2).zip


'tabnet_model(1.5w2).zip'

In [7]:
loaded_model = TabNetClassifier()


loaded_model.load_model("tabnet_model(1.5w2).zip")

In [8]:
scaler = StandardScaler()

X_train_f, X_val_f, y_train_f, y_val_f = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_train_f_scaled = scaler.fit_transform(X_train_f)
X_val_f_scaled   = scaler.transform(X_val_f)
X_test_new_scaled  = scaler.fit_transform(X_test_new)


X_train_scaled_np = X_train_f_scaled.astype(np.float32)
y_train_np = y_train_f.values.astype(np.int64)

X_val_scaled_np = X_val_f_scaled.astype(np.float32)
y_val_np = y_val_f.values.astype(np.int64)

X_test_new_scaled_np = X_test_new_scaled.astype(np.float32)
y_test_new_np = y_test_new.values.astype(np.int64)

y_score_tab = loaded_model.predict_proba(X_test_new_scaled_np)[:, 1]



df_scores = pd.read_csv("../roc_scores_rf.csv")   




df_scores["tab"] = y_score_tab


df_scores.to_csv("../roc_scores_rf.csv", index=False)




In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]   

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_fold)
    X_val_scaled   = scaler.transform(X_val_fold)

    
    X_train_fold = X_train_scaled.astype(np.float32)
    X_val_fold   = X_val_scaled.astype(np.float32)
    y_train_fold = y_train_fold.values.astype(np.int64)
    y_val_fold   = y_val_fold.values.astype(np.int64)

    model = TabNetClassifier(
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=0.003399481210795565),
        scheduler_params={"step_size":10, "gamma":0.9},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        verbose=10,
        n_d=64, n_a=48, n_steps=8, gamma=1.0205844942958024,
        lambda_sparse=8.706020878304853e-05,
        seed=42
    )

    model.fit(
        X_train=X_train_fold, y_train=y_train_fold,
        batch_size=1024,
        virtual_batch_size=128,
        max_epochs=33,
        num_workers=4
    )

    y_pred_val = model.predict(X_val_fold)
    acc = accuracy_score(y_val_fold, y_pred_val)

    y_train_pred = tabnet_model.predict(X_train_fold)
    train_acc = accuracy_score(y_train_fold, y_train_pred)

    
    print(f"TabNet Train Accuracy: {train_acc:.4f}")


    print(f"\n TabNet Test Accuracy: {acc:.4f}")
    


   


